In [ ]:
# 0. Import necessary libraries
!pip install clickhouse_connect
!pip install plotly
!pip install folium
!pip install geopandas

In [ ]:
# 1. Establish connection

import clickhouse_connect
client = clickhouse_connect.get_client(host='clickhouse', port=8123, database='NYC')
client.command('SELECT timezone()')

In [ ]:
# 2. Test query
import pandas as pd
result = client.query('SELECT count() FROM NYC.trips')
df = pd.DataFrame(result.result_rows)
df.columns = result.column_names
print(f"Total number of records in the trips table: {df['count()'][0]}")


In [ ]:
#2a. Test query
import pandas as pd
sample_query = '''
SELECT * FROM NYC.trips LIMIT 5
'''
result = client.query(sample_query)
sample_df = pd.DataFrame(result.result_rows, 
                        columns=result.column_names)
sample_df

In [ ]:
# 3. Import necessary libraries and create an utility function
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def query_to_df(query):
    """Execute a query and return the results as a DataFrame."""
    result = client.query(query)
    return pd.DataFrame(result.result_rows, 
                        columns=result.column_names)

In [ ]:
#4. Analysis by Trip Length
# Let's analyze the relationship between trip length, tips, fares, and passenger count:
trip_length_query = '''
SELECT 
    avg(tip_amount) AS avg_tip, 
    avg(fare_amount) AS avg_fare, 
    avg(passenger_count) AS avg_passenger, 
    count() AS count,
    truncate(date_diff('second', pickup_datetime, dropoff_datetime)/60) as trip_minutes
FROM NYC.trips
WHERE date_diff('second', pickup_datetime, dropoff_datetime) > 0
  AND trip_minutes <= 60  -- Limit to trips under an hour for better visualization
GROUP BY trip_minutes
ORDER BY trip_minutes
'''

trip_length_df = query_to_df(trip_length_query)
trip_length_df

In [ ]:
#Now, let's visualize this data to see the relationships:

# Create a multi-line plot using Plotly for better interactivity
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=trip_length_df['trip_minutes'], y=trip_length_df['avg_fare'],
               name="Average Fare", line=dict(color='blue')),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=trip_length_df['trip_minutes'], y=trip_length_df['avg_tip'],
               name="Average Tip", line=dict(color='green')),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=trip_length_df['trip_minutes'], y=trip_length_df['avg_passenger'],
               name="Avg Passengers", line=dict(color='orange')),
    secondary_y=True,
)

# Set titles
fig.update_layout(
    title_text="Trip Metrics by Trip Length",
    width=900,
    height=500
)

fig.update_xaxes(title_text="Trip Length (minutes)")
fig.update_yaxes(title_text="Amount ($)", secondary_y=False)
fig.update_yaxes(title_text="Average Passengers", secondary_y=True)

fig.show()

# Create another view showing the trip count by duration
plt.figure(figsize=(12, 6))
plt.bar(trip_length_df['trip_minutes'], trip_length_df['count'], color='purple', alpha=0.7)
plt.title('Number of Trips by Trip Length')
plt.xlabel('Trip Length (minutes)')
plt.ylabel('Number of Trips')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 5. Pickup Patterns by Neighborhood and Hour
# Let's analyze pickup patterns by neighborhood and hour:

pickup_pattern_query = '''
SELECT
    pickup_ntaname,
    toHour(pickup_datetime) as pickup_hour,
    count() AS pickups
FROM trips
WHERE pickup_ntaname != ''
GROUP BY pickup_ntaname, pickup_hour
ORDER BY pickup_ntaname, pickup_hour
'''

pickup_patterns_df = query_to_df(pickup_pattern_query)

# Get the top 10 neighborhoods by total pickups
top_neighborhoods_query = '''
SELECT
    pickup_ntaname,
    count() AS total_pickups
FROM trips
WHERE pickup_ntaname != ''
GROUP BY pickup_ntaname
ORDER BY total_pickups DESC
LIMIT 10
'''

top_neighborhoods = query_to_df(top_neighborhoods_query)['pickup_ntaname'].tolist()

# Filter the pickup patterns to only include top neighborhoods
top_patterns_df = pickup_patterns_df[pickup_patterns_df['pickup_ntaname'].isin(top_neighborhoods)]

# Create a pivot table for better visualization
pivot_df = top_patterns_df.pivot(index='pickup_hour', columns='pickup_ntaname', values='pickups')
pivot_df

In [ ]:
#Now, let's visualize the pickup patterns:


# Create a heatmap of pickups by neighborhood and hour
plt.figure(figsize=(14, 8))
sns.heatmap(pivot_df, cmap='viridis', annot=False, fmt='g')
plt.title('Pickups by Neighborhood and Hour of Day')
plt.xlabel('Neighborhood')
plt.ylabel('Hour of Day')
plt.tight_layout()
plt.show()

# Create line charts for each top neighborhood showing pickups by hour
plt.figure(figsize=(14, 8))
for neighborhood in top_neighborhoods:
    neighborhood_data = top_patterns_df[top_patterns_df['pickup_ntaname'] == neighborhood]
    plt.plot(neighborhood_data['pickup_hour'], neighborhood_data['pickups'], marker='o', label=neighborhood)

plt.title('Pickup Patterns Throughout the Day by Neighborhood')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Pickups')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 6. Geographic Analysis
# Let's visualize pickup and dropoff locations on a map:
# Get a sample of pickup and dropoff coordinates
geo_query = '''
SELECT 
    pickup_longitude, 
    pickup_latitude, 
    dropoff_longitude, 
    dropoff_latitude,
    fare_amount
FROM trips
WHERE pickup_longitude != 0 
  AND pickup_latitude != 0
  AND dropoff_longitude != 0
  AND dropoff_latitude != 0
  AND pickup_longitude > -75 AND pickup_longitude < -73
  AND pickup_latitude > 40 AND pickup_latitude < 42
  AND dropoff_longitude > -75 AND dropoff_longitude < -73
  AND dropoff_latitude > 40 AND dropoff_latitude < 42
LIMIT 5000
'''

geo_df = query_to_df(geo_query)

# Create a simple scatter plot of pickup locations colored by fare amount
plt.figure(figsize=(12, 10))
scatter = plt.scatter(
    geo_df['pickup_longitude'], 
    geo_df['pickup_latitude'],
    c=geo_df['fare_amount'],
    cmap='viridis',
    alpha=0.6,
    s=30
)

plt.colorbar(scatter, label='Fare Amount ($)')
plt.title('Pickup Locations by Fare Amount')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 7. Advanced Analysis: Fare Analysis by Time of Day and Day of Week

time_analysis_query = '''
SELECT
    toHour(pickup_datetime) as hour_of_day,
    toDayOfWeek(pickup_datetime) as day_of_week,
    avg(fare_amount) as avg_fare,
    avg(tip_amount) as avg_tip,
    count() as trip_count
FROM trips
GROUP BY hour_of_day, day_of_week
ORDER BY day_of_week, hour_of_day
'''

time_analysis_df = query_to_df(time_analysis_query)

# Create day of week labels (1 = Monday, 7 = Sunday in ClickHouse)
day_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
time_analysis_df['day_name'] = time_analysis_df['day_of_week'].apply(lambda x: day_labels[x-1])

# Create a pivot table for the heatmap
fare_pivot = time_analysis_df.pivot(index="day_name", columns="hour_of_day", values="avg_fare")
fare_pivot = fare_pivot.reindex(day_labels)  # Ensure correct order of days

# Create a heatmap of average fares by day and hour
plt.figure(figsize=(14, 8))
sns.heatmap(fare_pivot, cmap="YlGnBu", annot=True, fmt=".2f", linewidths=.5)
plt.title('Average Fare by Day of Week and Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Day of Week')
plt.tight_layout()
plt.show()

In [ ]:
# 8. Payment Type Analysis
payment_query = '''
SELECT
    payment_type,
    count() as trip_count,
    avg(fare_amount) as avg_fare,
    avg(tip_amount) as avg_tip,
    avg(tip_amount) / nullif(avg(fare_amount), 0) * 100 as tip_percentage
FROM trips
WHERE payment_type IN ('CSH', 'CRE')  -- Cash or Credit
GROUP BY payment_type
'''

payment_df = query_to_df(payment_query)

# Create a bar chart comparing payment types with matplotlib
fig, ax1 = plt.subplots(figsize=(10, 6))

# Set the positions and width for the bars
pos = list(range(len(payment_df['payment_type'])))
width = 0.3

# Plot bars for avg_fare and avg_tip
ax1.bar([p - width/2 for p in pos], payment_df['avg_fare'], width, color='skyblue', label='Average Fare')
ax1.bar([p + width/2 for p in pos], payment_df['avg_tip'], width, color='lightgreen', label='Average Tip')
ax1.set_ylabel('Amount ($)')
ax1.set_xticks(pos)
ax1.set_xticklabels(payment_df['payment_type'])
ax1.legend(loc='upper left')

# Create second y-axis for tip percentage
ax2 = ax1.twinx()
ax2.plot(pos, payment_df['tip_percentage'], 'ro-', linewidth=2, markersize=10, label='Tip Percentage')
ax2.set_ylabel('Tip Percentage (%)')
ax2.legend(loc='upper right')

plt.title('Payment Analysis by Type')
plt.xlabel('Payment Type')
plt.tight_layout()
plt.show()

In [ ]:
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

# Get a sample of pickup and dropoff coordinates
geo_query = '''
SELECT 
    pickup_longitude, 
    pickup_latitude, 
    dropoff_longitude, 
    dropoff_latitude,
    fare_amount
FROM trips
WHERE pickup_longitude != 0 
  AND pickup_latitude != 0
  AND dropoff_longitude != 0
  AND dropoff_latitude != 0
  AND pickup_longitude > -75 AND pickup_longitude < -73
  AND pickup_latitude > 40 AND pickup_latitude < 42
  AND dropoff_longitude > -75 AND dropoff_longitude < -73
  AND dropoff_latitude > 40 AND dropoff_latitude < 42
LIMIT 5000
'''
geo_df = query_to_df(geo_query)

# Calculate the center of the map
map_center = [geo_df['pickup_latitude'].mean(), geo_df['pickup_longitude'].mean()]

# Create a base map
nyc_map = folium.Map(location=map_center, zoom_start=11, tiles='CartoDB positron')

# Option 1: Create a heatmap of pickup locations
pickup_data = geo_df[['pickup_latitude', 'pickup_longitude']].values.tolist()
HeatMap(pickup_data, radius=15).add_to(nyc_map)

# Save and display the heatmap
nyc_map.save('nyc_taxi_heatmap.html')
display(nyc_map)

# Option 2: Create a map with clustered markers colored by fare amount
nyc_map2 = folium.Map(location=map_center, zoom_start=11, tiles='CartoDB positron')

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(nyc_map2)

# Define a function to determine marker color based on fare amount
def get_color(fare):
    if fare < 10:
        return 'green'
    elif fare < 20:
        return 'orange'
    else:
        return 'red'

# Add markers for each pickup location
for idx, row in geo_df.iterrows():
    # Limit to a smaller subset for better performance
    if idx % 20 == 0:  # Only plot every 20th point for better performance
        popup_text = f"Fare: ${row['fare_amount']:.2f}"
        folium.CircleMarker(
            location=[row['pickup_latitude'], row['pickup_longitude']],
            radius=5,
            popup=popup_text,
            fill=True,
            color=get_color(row['fare_amount']),
            fill_opacity=0.7
        ).add_to(marker_cluster)

# Add a legend (as a simple HTML element)
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: 90px; 
            border:2px solid grey; z-index:9999; background-color:white;
            padding: 10px;
            font-size: 14px;
            ">
  <span style="color:green;">■</span> < $10<br>
  <span style="color:orange;">■</span> $10-$20<br>
  <span style="color:red;">■</span> > $20<br>
</div>
'''
nyc_map2.get_root().html.add_child(folium.Element(legend_html))

# Save and display the clustered map
nyc_map2.save('nyc_taxi_clusters.html')
display(nyc_map2)

# Option 3: Show pickup to dropoff flows
nyc_map3 = folium.Map(location=map_center, zoom_start=11, tiles='CartoDB positron')

# Add a small sample of pickup-dropoff pairs as lines
for idx, row in geo_df.iterrows():
    if idx % 100 == 0 and idx < 500:  # Only plot a few lines to avoid clutter
        # Draw a line between pickup and dropoff
        locations = [
            [row['pickup_latitude'], row['pickup_longitude']],
            [row['dropoff_latitude'], row['dropoff_longitude']]
        ]
        folium.PolyLine(
            locations,
            color='blue',
            weight=2,
            opacity=0.7,
            popup=f"Fare: ${row['fare_amount']:.2f}"
        ).add_to(nyc_map3)
        
        # Add markers for pickup and dropoff
        folium.CircleMarker(
            location=[row['pickup_latitude'], row['pickup_longitude']],
            radius=3,
            color='green',
            fill=True,
            fill_opacity=0.7,
            popup='Pickup'
        ).add_to(nyc_map3)
        
        folium.CircleMarker(
            location=[row['dropoff_latitude'], row['dropoff_longitude']],
            radius=3,
            color='red',
            fill=True,
            fill_opacity=0.7,
            popup='Dropoff'
        ).add_to(nyc_map3)

# Save and display the flow map
nyc_map3.save('nyc_taxi_flows.html')
display(nyc_map3)